# Ai-Saturday Chatbot

![title](images/GenerativeChatBots.png)

## Background Information 

Just as the name suggests, generative chatbots create their own responses from scratch. They are trained from a set of conversation pairs and the chatbot will be able to construct a new sentence based on the question it is asked.

However, some challenges with these kinds of chatbots are:
1. They are prone to errors - they need to take the spelling and grammar into account
2. Building a generative chatbot requires skills and machine resources and alot of data

Once trained, the generative chatbots outperform their rule-based models as they can answer complex and unseen  queries :-)

### Sequence to Sequence Modelling (Seq2Seq) 

A Sequence to Sequence Model is one that has two Recurrent Neural Networks (RNN), known as Encoder and Decoder. The Encoder maps a variable variable-length source sequence into a fixed-length vector and the Decoder maps the vector representation back to a variable-length target sequence.

These two networks are then trained jointly to maximize the conditional probability of the target sequence given a source sequence.

An RNN is used because the output of an RNN is influenced by a series of past inputs as the contexts of next words does not necessarily rely on the previous word but to a series of words before that.

![title](images/rnn_tasks.png)

Depending on the problem being solved, an RNN can be used to deal with various tasks. Refering to the above diagram, each rectangle is a vector and arrows represent functions (e.g. Matrix Multiplication). The input vectors are red, output vectors are in blud and green vectors hold the RNN's state. Some of the tasks that can be done by an RNN are:
1. **one-to-one** - This is considered as the Vanilla Mode of processing without RNN. From fixed-size input to fixed-size output e.g. Image classification problem
2. **one-to-many** - This is considered as a sequence output e.g. image captioning, the network takes an image and outputs a sequence of words.
3. **many-to-one** - This is considered as a sequence input e.g. sentiment analysis where a given sentence is classified as either positive, negative or neutral.
4. **many-to-many** - This is considered as a sequence input and sequence output e.g. Machine Translation, the RNN reads a sentence in English and Translates it to a different language e.g French
5. **many-to-many** - This is an example of a synced sequence input and output e.g. video classification where there is need to label each frame of the video



### LSTMs - Long Short Term Memory 

The vanila mode RNN faces problems with vanishing gradient which paved the way for LSTMs that handles the problem very well due to the introduction of memory cells and gates.

**Vanishing gradient** - 
When training of Artificial Neural Networks using gradient based learning methods e.g. SGD aka Stochastic Gradient Descent which is an iterative method for optimizing an objective function using randomly selected samples to evaluate gradients, and backpropagation, there are instances when the gradient is vanishingly small effectively preventing the weight from changing its value essentially stoping the neural network from further training/learning.

LSTMs are a type of RNN that handles the long term dependencies problem of RNNs very well due to the introduction of gates in LSTM. It allows the cells to remember what information needs to be remembered from the previous cells and what needs to be updated.

The Encoders and Decoders in a Seq2Seq RNN are LSTMs

In [2]:
#Import the basic files
import pandas as pd
import numpy as np


#Import the necessary libraries
import os
import json
import nltk
import gensim

from gensim import corpora,models,similarities
import pickle
import nltk
nltk.download('punkt')

# Import Keras Libraries
from keras.models import Sequential
from keras.layers.recurrent import LSTM,SimpleRNN
import theano
theano.config.optimizer="None"

# Import SK-Learn Libraries
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.
W0909 00:31:00.492530 140331093419840 blas_headers.py:988] Using NumPy C-API based implementation for BLAS functions.


In [3]:
print("Gensim Version: ",gensim.__version__)

Gensim Version:  3.8.0


In [4]:
#import a pre-trained word-to-vec model
model = gensim.models.Word2Vec.load('models/word2vec.bin')

In [5]:
#Import the necessary Files
conversations_df = pd.read_csv("data/AiSaturdayChatbot.csv",engine="python")
conversations_df

,Query,Answer
0,Hi,"Hi to you, how are you doing?"
1,"Hi to you, how are you doing?",I am fine thank you. So what is your name?
2,I am fine thank you. So what is your name?,Oooh good to know. My name is AiSaturday Chatb...
3,What is AiSaturday all about,"Great, AiSaturday is about a group of students..."
4,"Great, AiSaturday is about a group of students...",I am a member thanks. This is great thank you.
5,Ok Cool. Have a great day sir/maadam,You too. Have a wonderful day


In [6]:
#Create the lists
x = conversations_df["Query"]
y = conversations_df["Answer"]

In [7]:
def tokeniz(query):    
    return nltk.word_tokenize(query.lower())

In [8]:
#Tokenize the lists
conversations_df['tok_x'] = conversations_df['Query'].apply(tokeniz)
conversations_df['tok_y'] = conversations_df['Answer'].apply(tokeniz)
conversations_df

,Query,Answer,tok_x,tok_y
0,Hi,"Hi to you, how are you doing?",[hi],"[hi, to, you, ,, how, are, you, doing, ?]"
1,"Hi to you, how are you doing?",I am fine thank you. So what is your name?,"[hi, to, you, ,, how, are, you, doing, ?]","[i, am, fine, thank, you, ., so, what, is, you..."
2,I am fine thank you. So what is your name?,Oooh good to know. My name is AiSaturday Chatb...,"[i, am, fine, thank, you, ., so, what, is, you...","[oooh, good, to, know, ., my, name, is, aisatu..."
3,What is AiSaturday all about,"Great, AiSaturday is about a group of students...","[what, is, aisaturday, all, about]","[great, ,, aisaturday, is, about, a, group, of..."
4,"Great, AiSaturday is about a group of students...",I am a member thanks. This is great thank you.,"[great, ,, aisaturday, is, about, a, group, of...","[i, am, a, member, thanks, ., this, is, great,..."
5,Ok Cool. Have a great day sir/maadam,You too. Have a wonderful day,"[ok, cool, ., have, a, great, day, sir/maadam]","[you, too, ., have, a, wonderful, day]"


In [9]:
tok_x = list(conversations_df['tok_x'])
tok_y = list(conversations_df['tok_y'])

In [10]:
# Create a vector of one's so that it can be appended at the end of the sentence
sentend = np.ones((300,),dtype=np.float32)

In [11]:
# Create word vectors for the words in the list tokens
vec_x = []
for sent in tok_x:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_x.append(sentvec)
    
vec_y = []
for sent in tok_y:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_y.append(sentvec)


/usr/local/gcti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/usr/local/gcti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [12]:
# Clip only the first 14 character length by romizing all the words after this. The objective is to make all the sizes the same
for tok_sent in vec_x:
    tok_sent[14:]=[]
    tok_sent.append(sentend)
    
# Append sentend to the sentence so as to make the size of the sentence to be 15
for tok_sent in vec_x:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)
            
for tok_sent in vec_y:
    tok_sent[14:]=[]
    tok_sent.append(sentend)
            
for tok_sent in vec_y:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)

# Save the pickle dump
word_vectors = [vec_x,vec_y]
with open('pickles/conversation.pickle','wb') as f:
    pickle.dump(word_vectors,f)

In [13]:
# Load the saved pickle file
with open('pickles/conversation.pickle','rb') as seq_f:
    seq_vec_x,seq_vec_y=pickle.load(seq_f)

In [14]:
seq2seq_vec_x=np.array(seq_vec_x,dtype=np.float64)
seq2seq_vec_y=np.array(seq_vec_y,dtype=np.float64)

In [15]:
X_train,x_test,y_train,y_test = train_test_split(seq2seq_vec_x,seq2seq_vec_y,test_size=0.2,random_state=2)

In [16]:
seq_model=Sequential()

W0909 00:31:33.571527 140331093419840 deprecation_wrapper.py:119] From /usr/local/gcti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [17]:
# Define the layers
seq_model.add(LSTM(output_dim=300,input_shape=X_train.shape[1:],return_sequences=True,init='glorot_normal',inner_init='glorot_normal',activation='sigmoid'))
seq_model.add(LSTM(output_dim=300,input_shape=X_train.shape[1:],return_sequences=True,init='glorot_normal',inner_init='glorot_normal',activation='sigmoid'))
seq_model.add(LSTM(output_dim=300,input_shape=X_train.shape[1:],return_sequences=True,init='glorot_normal',inner_init='glorot_normal',activation='sigmoid'))
seq_model.add(LSTM(output_dim=300,input_shape=X_train.shape[1:],return_sequences=True,init='glorot_normal',inner_init='glorot_normal',activation='sigmoid'))

# Loss Function
seq_model.compile(loss='cosine_proximity',optimizer='adam',metrics=['accuracy'])

/usr/local/gcti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  
W0909 00:31:33.814491 140331093419840 deprecation_wrapper.py:119] From /usr/local/gcti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0909 00:31:33.871515 140331093419840 deprecation_wrapper.py:119] From /usr/local/gcti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

/usr/local/gcti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), return_sequences=True, ac

In [18]:
# Fit the model in steps of 500
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM500.h5')
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM1000.h5')
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM1500.h5')
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM2000.h5')
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM2500.h5')
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM3000.h5')
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM3500.h5')
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM4000.h5')
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM4500.h5')
seq_model.fit(X_train,y_train,epochs=500,validation_data=(x_test,y_test))
seq_model.save('LSTM5000.h5')

W0909 00:31:35.991498 140331093419840 deprecation.py:323] From /usr/local/gcti/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0909 00:31:41.638763 140331093419840 deprecation_wrapper.py:119] From /usr/local/gcti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0909 00:31:42.085436 140331093419840 deprecation_wrapper.py:119] From /usr/local/gcti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 4 samples, validate on 2 samples
Epoch 1/500
4/4 [==============================] - 14s 3s/step - loss: -0.2606 - acc: 0.0000e+00 - val_loss: -0.2356 - val_acc: 0.0000e+00
Epoch 2/500
4/4 [==============================] - 3s 681ms/step - loss: -0.2938 - acc: 0.0167 - val_loss: -0.2736 - val_acc: 0.0000e+00
Epoch 3/500
4/4 [==============================] - 2s 473ms/step - loss: -0.3270 - acc: 0.0167 - val_loss: -0.3085 - val_acc: 0.0000e+00
Epoch 4/500
4/4 [==============================] - 2s 460ms/step - loss: -0.3562 - acc: 0.0167 - val_loss: -0.3389 - val_acc: 0.0000e+00
Epoch 5/500
4/4 [==============================] - 2s 503ms/step - loss: -0.3805 - acc: 0.0167 - val_loss: -0.3647 - val_acc: 0.0000e+00
Epoch 6/500
4/4 [==============================] - 2s 399ms/step - loss: -0.4004 - acc: 0.0167 - val_loss: -0.3864 - val_acc: 0.0000e+00
Epoch 7/500
4/4 [==============================] - 2s 480ms/step - loss: -0.4167 - acc: 0.0167 - val_loss: -0.4043 - val_acc: 0.0000e+

Epoch 61/500
4/4 [==============================] - 1s 140ms/step - loss: -0.4951 - acc: 0.0333 - val_loss: -0.4690 - val_acc: 0.0333
Epoch 62/500
4/4 [==============================] - 1s 133ms/step - loss: -0.4959 - acc: 0.0333 - val_loss: -0.4700 - val_acc: 0.0000e+00
Epoch 63/500
4/4 [==============================] - 1s 131ms/step - loss: -0.4968 - acc: 0.0333 - val_loss: -0.4708 - val_acc: 0.0000e+00
Epoch 64/500
4/4 [==============================] - 1s 141ms/step - loss: -0.4975 - acc: 0.0333 - val_loss: -0.4712 - val_acc: 0.0333
Epoch 65/500
4/4 [==============================] - 1s 146ms/step - loss: -0.4982 - acc: 0.0333 - val_loss: -0.4715 - val_acc: 0.0333
Epoch 66/500
4/4 [==============================] - 1s 132ms/step - loss: -0.4988 - acc: 0.0500 - val_loss: -0.4717 - val_acc: 0.0000e+00
Epoch 67/500
4/4 [==============================] - 1s 137ms/step - loss: -0.4994 - acc: 0.0500 - val_loss: -0.4718 - val_acc: 0.0000e+00
Epoch 68/500
4/4 [============================

Epoch 121/500
4/4 [==============================] - 1s 137ms/step - loss: -0.5173 - acc: 0.0167 - val_loss: -0.4752 - val_acc: 0.0333
Epoch 122/500
4/4 [==============================] - 1s 131ms/step - loss: -0.5175 - acc: 0.0167 - val_loss: -0.4757 - val_acc: 0.0000e+00
Epoch 123/500
4/4 [==============================] - 1s 142ms/step - loss: -0.5177 - acc: 0.0167 - val_loss: -0.4750 - val_acc: 0.0000e+00
Epoch 124/500
4/4 [==============================] - 1s 145ms/step - loss: -0.5173 - acc: 0.0333 - val_loss: -0.4739 - val_acc: 0.0000e+00
Epoch 125/500
4/4 [==============================] - 1s 156ms/step - loss: -0.5168 - acc: 0.0167 - val_loss: -0.4739 - val_acc: 0.0000e+00
Epoch 126/500
4/4 [==============================] - 1s 125ms/step - loss: -0.5165 - acc: 0.0333 - val_loss: -0.4728 - val_acc: 0.0000e+00
Epoch 127/500
4/4 [==============================] - 1s 136ms/step - loss: -0.5180 - acc: 0.0333 - val_loss: -0.4706 - val_acc: 0.0333
Epoch 128/500
4/4 [================

Epoch 181/500
4/4 [==============================] - 1s 131ms/step - loss: -0.5311 - acc: 0.0333 - val_loss: -0.4743 - val_acc: 0.0000e+00
Epoch 182/500
4/4 [==============================] - 1s 131ms/step - loss: -0.5314 - acc: 0.0167 - val_loss: -0.4747 - val_acc: 0.0000e+00
Epoch 183/500
4/4 [==============================] - 1s 137ms/step - loss: -0.5316 - acc: 0.0167 - val_loss: -0.4745 - val_acc: 0.0000e+00
Epoch 184/500
4/4 [==============================] - 1s 135ms/step - loss: -0.5318 - acc: 0.0167 - val_loss: -0.4734 - val_acc: 0.0000e+00
Epoch 185/500
4/4 [==============================] - 1s 127ms/step - loss: -0.5320 - acc: 0.0167 - val_loss: -0.4738 - val_acc: 0.0000e+00
Epoch 186/500
4/4 [==============================] - 1s 142ms/step - loss: -0.5322 - acc: 0.0333 - val_loss: -0.4728 - val_acc: 0.0000e+00
Epoch 187/500
4/4 [==============================] - 1s 137ms/step - loss: -0.5322 - acc: 0.0167 - val_loss: -0.4763 - val_acc: 0.0000e+00
Epoch 188/500
4/4 [========

...

Epoch 222/500
4/4 [==============================] - 1s 135ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4091 - val_acc: 0.0667
Epoch 223/500
4/4 [==============================] - 1s 132ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4091 - val_acc: 0.0667
Epoch 224/500
4/4 [==============================] - 0s 120ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4091 - val_acc: 0.0667
Epoch 225/500
4/4 [==============================] - 1s 141ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4091 - val_acc: 0.1000
Epoch 226/500
4/4 [==============================] - 1s 133ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4091 - val_acc: 0.1000
Epoch 227/500
4/4 [==============================] - 1s 141ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4090 - val_acc: 0.0667
Epoch 228/500
4/4 [==============================] - 1s 144ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4090 - val_acc: 0.0667
Epoch 229/500
4/4 [==============================] - 1s

Epoch 283/500
4/4 [==============================] - 1s 141ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4088 - val_acc: 0.0667
Epoch 284/500
4/4 [==============================] - 1s 134ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4089 - val_acc: 0.0667
Epoch 285/500
4/4 [==============================] - 1s 136ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4089 - val_acc: 0.0667
Epoch 286/500
4/4 [==============================] - 1s 136ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4089 - val_acc: 0.0667
Epoch 287/500
4/4 [==============================] - 1s 131ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4089 - val_acc: 0.0667
Epoch 288/500
4/4 [==============================] - 1s 147ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4090 - val_acc: 0.0667
Epoch 289/500
4/4 [==============================] - 0s 124ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4089 - val_acc: 0.0667
Epoch 290/500
4/4 [==============================] - 1s

Epoch 344/500
4/4 [==============================] - 1s 144ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4088 - val_acc: 0.0667
Epoch 345/500
4/4 [==============================] - 1s 126ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4086 - val_acc: 0.0667
Epoch 346/500
4/4 [==============================] - 0s 124ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4086 - val_acc: 0.0667
Epoch 347/500
4/4 [==============================] - 1s 146ms/step - loss: -0.6552 - acc: 0.0500 - val_loss: -0.4087 - val_acc: 0.1000
Epoch 348/500
4/4 [==============================] - 1s 144ms/step - loss: -0.6552 - acc: 0.0500 - val_loss: -0.4087 - val_acc: 0.0667
Epoch 349/500
4/4 [==============================] - 1s 135ms/step - loss: -0.6552 - acc: 0.0500 - val_loss: -0.4087 - val_acc: 0.0667
Epoch 350/500
4/4 [==============================] - 1s 138ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4087 - val_acc: 0.0667
Epoch 351/500
4/4 [==============================] - 1s

Epoch 405/500
4/4 [==============================] - 1s 144ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4085 - val_acc: 0.0667
Epoch 406/500
4/4 [==============================] - 1s 127ms/step - loss: -0.6552 - acc: 0.0500 - val_loss: -0.4086 - val_acc: 0.0667
Epoch 407/500
4/4 [==============================] - 0s 122ms/step - loss: -0.6552 - acc: 0.0500 - val_loss: -0.4086 - val_acc: 0.0667
Epoch 408/500
4/4 [==============================] - 1s 129ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4086 - val_acc: 0.0667
Epoch 409/500
4/4 [==============================] - 1s 135ms/step - loss: -0.6552 - acc: 0.0500 - val_loss: -0.4086 - val_acc: 0.0667
Epoch 410/500
4/4 [==============================] - 1s 130ms/step - loss: -0.6552 - acc: 0.0333 - val_loss: -0.4085 - val_acc: 0.0667
Epoch 411/500
4/4 [==============================] - 1s 132ms/step - loss: -0.6552 - acc: 0.0500 - val_loss: -0.4085 - val_acc: 0.0667
Epoch 412/500
4/4 [==============================] - 1s

Epoch 466/500
4/4 [==============================] - 1s 145ms/step - loss: -0.6553 - acc: 0.0333 - val_loss: -0.4083 - val_acc: 0.0667
Epoch 467/500
4/4 [==============================] - 1s 152ms/step - loss: -0.6553 - acc: 0.0333 - val_loss: -0.4083 - val_acc: 0.1000
Epoch 468/500
4/4 [==============================] - 1s 156ms/step - loss: -0.6553 - acc: 0.0333 - val_loss: -0.4083 - val_acc: 0.0667
Epoch 469/500
4/4 [==============================] - 1s 131ms/step - loss: -0.6553 - acc: 0.0333 - val_loss: -0.4083 - val_acc: 0.0667
Epoch 470/500
4/4 [==============================] - 1s 127ms/step - loss: -0.6553 - acc: 0.0500 - val_loss: -0.4084 - val_acc: 0.0667
Epoch 471/500
4/4 [==============================] - 1s 146ms/step - loss: -0.6553 - acc: 0.0333 - val_loss: -0.4084 - val_acc: 0.0667
Epoch 472/500
4/4 [==============================] - 1s 145ms/step - loss: -0.6553 - acc: 0.0333 - val_loss: -0.4084 - val_acc: 0.0667
Epoch 473/500
4/4 [==============================] - 1s

In [19]:
#Run the predictions
predictions = seq_model.predict(x_test)

In [20]:
#Show the results
mod = gensim.models.Word2Vec.load('models/word2vec.bin')
#[mod.most_similar([predictions[10][i]])[0] for i in range(15)]

MemoryError: 